In [1]:
import redis
import pandas as pd

In [10]:
redis_client = redis.Redis(host='localhost', port=6379, db=0)

In [17]:
def read_cached_df(path):
    serialized_df = redis_client.get(path)
    if serialized_df:
      serialized_df = serialized_df.decode('utf-8')
      cached_df = pd.read_json(serialized_df)
      return cached_df
    else:
      raise FileNotFoundError(f"No cached DataFrame found at path: {path}")
    
df = read_cached_df("pos_ev_b5b20918314d95ea0d34ea75f534f3b4")
print(df)

                                               outcome  fanduel  \
19   alternate_spreads_1st_5_innings_Atlanta Braves...      0.0   
162              batter_home_runs_Over_Trea Turner_0.5     21.0   
160               batter_home_runs_Over_Matt Olson_0.5     17.0   
22   alternate_spreads_1st_5_innings_Philadelphia P...      0.0   
233          batter_total_bases_Over_J.T. Realmuto_1.5      4.7   
..                                                 ...      ...   
305                 batter_rbis_Over_Marcell Ozuna_0.5      0.0   
325                   batter_rbis_Over_Gio Urshela_0.5      0.0   
317                batter_rbis_Over_Ramón Laureano_1.5      0.0   
331                   batter_rbis_Over_Johan Rojas_0.5      0.0   
548            alternate_totals_1st_5_innings_Over_7.5      0.0   

                          market_key           outcome_name  \
19   alternate_spreads_1st_5_innings         Atlanta Braves   
162                 batter_home_runs                   Over   
160   

/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_4915/2444418874.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  cached_df = pd.read_json(serialized_df)


In [18]:
def get_cache_expiration(cache_name):
    """
    Get the expiration time of a cache entry in Redis.

    :param redis_client: Redis client instance
    :param cache_name: Name of the cache (key) to check
    :return: Remaining time-to-live in seconds, or None if the key does not exist or has no expiration
    """
    ttl = redis_client.ttl(cache_name)
    
    if ttl == -1:
        return None  # The key exists but does not have an expiration time
    elif ttl == -2:
        return None  # The key does not exist
    else:
        return ttl  # The remaining time-to-live in seconds

cache_name = "pos_ev_b5b20918314d95ea0d34ea75f534f3b4"

expiration_seconds = get_cache_expiration(cache_name)
print(expiration_seconds)

16361


In [20]:
16361 / 60 /60 

4.544722222222222

In [22]:
def write_to_cache(df, cache_path):
    
    if not df.index.is_unique:
        df = df.reset_index(drop=True)
    serialized_df = df.to_json()
    redis_client.set(cache_path, serialized_df)
    print(f"df written to: {cache_path}")
write_to_cache(pd.DataFrame(), 'americanfootball_ncaaf_pos_ev_cache')

df written to: americanfootball_ncaaf_pos_arb_cache


In [27]:
df = pd.read_csv("/Users/stefanfeiler/Desktop/PFL_10:_2023_PFL_World_ChampionshipNovember_23_November_23.csv")

In [21]:
def categorize_markets(df):
    def categorize(row):
        market = row['market'].lower()

        try:
          current_market_keys = row['market_key'].lower()
        except:
            current_market_keys = ['']

        if any(x in market for x in ['wins by']):
            return 'Method of Victory'
        
        if any(x in market for x in ['wins', 'draw']) or market in ['over', 'under'] or any(x in current_market_keys for x in ['h2h', 'totals']) or any(x in market for x in ['scorecards = no action', 'ends in a draw']):
            return 'Fight lines'
        
        if any(x in market for x in ['round', 'ends in round']):
            return 'Round props'
        
        
        
        return 'Other props'

    df['market_key'] = df.apply(categorize, axis=1)
    return df
new_df = categorize_markets(df)

In [31]:
def categorize_dropdown(df):
    def categorize(row):
        market = row['market'].lower()

        if 'wins by' in market and 'round' in market:
            return True
        else:
            return False

    df['dropdown'] = df.apply(categorize, axis=1)
    return df
new_df = categorize_dropdown(df)

In [32]:
new_df.to_csv("/Users/stefanfeiler/Desktop/MMA_NEW_TEST.csv", index= False)

In [33]:
new_df[['market', 'dropdown']]

,market,dropdown
0,Clay Collard,False
1,Olivier Aubin-Mercier,False
2,Fight ends in a Draw,False
3,Over 0.5 rounds,False
4,Under 0.5 rounds,False
...,...,...
399,Stirn wins in round 3,False
400,Blyden wins in round 3,False
401,Stirn wins by KO/TKO or Submission,False
402,Jesse Stirn (scorecards = no action),False


In [66]:
df = pd.read_csv("/Users/stefanfeiler/Desktop/UFC_Fight_Night:_Burns_vs._BradySeptember_6_September_6.csv")

In [67]:
desired_columns = [
    'DraftKings', 'BetMGM', 'Caesars', 'BetRivers', 'FanDuel', 'Bet365', 'Unibet', 'PointsBet', 
    'BetOnline', 'BetAnySports', 'BetUS', 'Cloudbet', 'Jazz', 'MyBookie', 'Pinnacle', 'SXBet', 
    'Bovada', 'Betway'
]

# Find which of the desired columns are actually present in the DataFrame
existing_columns = [col for col in desired_columns if col in df.columns]

# Compute the average only for the existing columns
df['avg_mkt_odds'] = df[existing_columns].mean(axis=1)

In [68]:
df['market']

0                                    Gilbert Burns
1                                       Sean Brady
2                             Fight ends in a Draw
3                                  Over 1.5 rounds
4                                 Under 1.5 rounds
                          ...                     
434          Fletcher wins in round 2 - Submission
435           Ramaska wins in round 3 - Submission
436          Fletcher wins in round 3 - Submission
437    Zygimantas Ramaska (scorecards = no action)
438       Nathan Fletcher (scorecards = no action)
Name: market, Length: 439, dtype: object

In [72]:
import pandas as pd
import numpy as np

# Assuming we have a DataFrame 'df' with columns: 'game_id', 'market', 'avg_mkt_odds'

# Step 1: Filter for 'Over' or 'Under' and 'Rounds' markets
def mark_main_totals(df):
    def is_over_under_rounds(market):
        return market.str.contains(r'(Over|Under)', regex=True)

    filtered_df = df[is_over_under_rounds(df['market'])].copy()

    # Step 2: Extract round value
    filtered_df['round_value'] = filtered_df['market'].str.extract(r'(\d+\.?\d*)')[0].astype(float)

    # Step 3: Group by game_id and round_value, then sum the avg_mkt_odds
    grouped = filtered_df.groupby(['game_id', 'round_value'])['avg_mkt_odds'].sum().reset_index()

    # Step 4: Find the round value with the lowest sum for each game
    lowest_rounds = grouped.loc[grouped.groupby('game_id')['avg_mkt_odds'].idxmin()]

    # Step 5: Merge this information back to the filtered DataFrame
    filtered_df = filtered_df.merge(lowest_rounds[['game_id', 'round_value']], 
                                    on=['game_id', 'round_value'], 
                                    how='left', 
                                    indicator=True)

    # Step 6: Mark the rows
    filtered_df['mark'] = np.where(filtered_df['_merge'] == 'both', 'Lowest Group', 'Other')
    filtered_df = filtered_df.drop('_merge', axis=1)

    # Step 7: Apply the marks to the original DataFrame
    df = df.merge(filtered_df[['game_id', 'market', 'mark']], 
                on=['game_id', 'market'], 
                how='left')

    # Fill NaN values in 'mark' column for rows that were not in the filtered DataFrame
    df['mark'] = df['mark'].fillna('Not Applicable')
    df = df.replace('Main Total Under', '')
    df = df.replace('Main Total Over', '')
    df['market_key'] = np.where(df['mark'] == 'Lowest Group', 'Main', df['market_key'])
    columns_to_drop = ['mark', 'mark_y', 'mark_x', 'avg_mkt_odds']
    for col in columns_to_drop:
        if col in df.columns:  # Check if the column exists before attempting to drop it
            df = df.drop(columns=col)
    return df

     Unnamed: 0            market  BetOnline    Bovada  Jazz  MyBookie  \
3             3   Over 1.5 rounds        NaN  1.153846   NaN       NaN   
4             4  Under 1.5 rounds        NaN  5.000000   NaN       NaN   
5             5   Over 2.5 rounds        NaN  1.322581   NaN       NaN   
6             6  Under 2.5 rounds        NaN  3.250000   NaN       NaN   
7             7   Over 3.5 rounds   1.454545  1.487805   NaN  1.476190   
8             8  Under 3.5 rounds   2.800000  2.550000   NaN  2.490000   
9             9   Over 4.5 rounds        NaN       NaN   NaN       NaN   
10           10  Under 4.5 rounds        NaN       NaN   NaN       NaN   
62           62   Over 1.5 rounds        NaN  1.307692   NaN       NaN   
63           63  Under 1.5 rounds        NaN  3.300000   NaN       NaN   
64           64   Over 2.5 rounds   1.571429  1.526316   NaN  1.523560   
65           65  Under 2.5 rounds   2.450000  2.450000   NaN  2.360000   
104         104   Over 1.5 rounds   2.

/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_92640/2072429332.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return market.str.contains(r'(Over|Under)', regex=True)


In [73]:
df.to_csv("/Users/stefanfeiler/Desktop/UFC_Fight_Night:_Burns_vs._BradySeptember_6_September_6_TEST.csv")